In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

In [2]:
import sklearn
sklearn.show_versions()


System:
    python: 3.8.5 (default, Sep  4 2020, 02:22:02)  [Clang 10.0.0 ]
executable: /Users/metra4ok/anaconda3/bin/python
   machine: macOS-10.15.7-x86_64-i386-64bit

Python dependencies:
          pip: 20.3.3
   setuptools: 52.0.0.post20210125
      sklearn: 0.24.1
        numpy: 1.19.2
        scipy: 1.5.2
       Cython: 0.29.21
       pandas: 1.2.1
   matplotlib: 3.3.2
       joblib: 1.0.0
threadpoolctl: 2.1.0

Built with OpenMP: True


In [3]:
import random

seed = 42
np.random.seed(seed)
random.seed(seed)

# Загрузка датасета

In [4]:
data = pd.read_csv('preprocessed_data.csv', index_col=0)

data.head()

,МАРКА_rare,МАРКА_Э76ХФ,МАРКА_Э90ХАФ,ПРОФИЛЬ_rare,ПРОФИЛЬ_Р65,t вып-обр,t обработка,t под током,t продувка,ПСН гр.,...,химшлак первый MnO_1,химшлак первый R_1,химшлак первый SiO2_1,химшлак последний Al2O3,химшлак последний CaO,химшлак последний FeO,химшлак последний MgO,химшлак последний MnO,химшлак последний R,химшлак последний SiO2
0,0,1,0,0,1,29.0,45.366667,24.400000,41.033333,3.004414,...,0.12,2.6,22.704914,NaN,53.4,1.0,5.8,0.15,2.6,NaN
1,0,1,0,0,1,26.0,44.066667,13.866667,44.066667,3.004414,...,0.14,2.2,22.704914,NaN,59.3,0.6,4.1,0.11,2.3,NaN
2,0,1,0,0,1,24.0,43.350000,17.950000,43.350000,3.004414,...,0.12,2.6,22.704914,NaN,57.2,0.5,8.1,0.13,2.3,NaN
3,0,1,0,0,1,17.0,46.183333,19.816667,46.183333,3.004414,...,0.11,2.4,22.704914,NaN,60.3,0.5,6.3,0.12,2.6,NaN
4,0,1,0,0,1,20.0,48.500000,17.033333,48.500000,3.004414,...,0.12,2.4,22.704914,NaN,60.3,0.6,4.5,0.09,2.5,NaN


In [5]:
data.shape

(6237, 72)

In [6]:
target_features = ['химшлак последний Al2O3', 'химшлак последний CaO',
                   'химшлак последний FeO', 'химшлак последний MgO',
                   'химшлак последний MnO', 'химшлак последний R',
                   'химшлак последний SiO2']

Для каждого таргета будем строить свою модель.

**У некоторых таргетов есть пропуски значений - их необходимо удалять**

# химшлак последний CaO

1. Разделение данных

In [7]:
target = 'химшлак последний CaO'
targets_to_drop = ['химшлак последний Al2O3', 'химшлак последний FeO',
                   'химшлак последний MgO', 'химшлак последний MnO',
                   'химшлак последний R', 'химшлак последний SiO2']
cleaned_data = data.drop(targets_to_drop, axis=1)
cleaned_data = cleaned_data.dropna()
X = cleaned_data.drop(target, axis=1)
y = cleaned_data[target]

2. Нормализация

In [8]:
X = StandardScaler().fit_transform(X)

3. Разделение на трейн и тест

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=seed)
print("Train size: {}, test size: {}".format(X_train.shape, X_test.shape))

Train size: (5613, 65), test size: (624, 65)


# Baseline

In [10]:
lin_reg_base = LinearRegression()
scores_lin_reg_base = cross_val_score(lin_reg_base, X_train, y_train, 
                                      scoring='neg_mean_absolute_percentage_error')
print("Кросс-валидация на трейне: {}".format(np.abs(scores_lin_reg_base.mean())))

lin_reg_base.fit(X_train, y_train)
test_pred_lin_reg_base = lin_reg_base.predict(X_test)
test_score_lin_reg_base = mean_absolute_percentage_error(test_pred_lin_reg_base, y_test)
print("Валидация на тесте: {}".format(test_score_lin_reg_base))

Кросс-валидация на трейне: 0.04784342137951148
Валидация на тесте: 0.04527818250538463
